# Foodies Rejoice! Visualising Hawker Centre Locations in Singapore

This notebook is a short preview into how one can visualise geographical data using folium that is prepared for the PyData Singapore Meetup - Visualising and Analyzing Big Data - October 2019.

The data used in this notebook is publicly available, and can be obtained from: https://data.gov.sg/dataset/hawker-centres

## Importing all our dependencies

In [ ]:
import folium
from folium import plugins
from dataclasses import dataclass, field
from typing import List
import base64
import json
import requests

## Defining the names of the files we will use

This assumes that the geojson files have been downloaded, and unzipped into a folder named `hawker-centres`, with the file `hawker-centres-geojson.geojson` inside. If you have not downloaded the dataset, download it first :)

In [ ]:
hawker_centre_geojson = "hawker-centres/hawker-centres-geojson.geojson"
truncated_hawker_centre_geojson = "hawker-centres/hawker-centres-truncated-geojson.geojson"

## Quick peek at our data

In [ ]:
def get_json_data(filename):
    with open(filename, "r") as f:
        return json.load(f)
        
get_json_data(truncated_hawker_centre_geojson)

## Helper Method

Get the all the Coordinates to plot in a file. (This is mainly used to dynamically fit the maps to all the coordinates to be plotted, more on this later)

In [ ]:
def get_all_coords_in_file(filename):
    features = get_json_data(filename)["features"]
    all_coords = [feature["geometry"]["coordinates"][:2][::-1] for feature in features]
    return all_coords

## Brief introduction to Folium

In folium, maps are hierarchical objects, you make a base map and then you add new items onto that object (make a canvas and add things to the canvas)

In [ ]:
def intro_to_folium():
    viz_map = folium.Map(
        location=[45.372, -121.6972],
        zoom_start=12,
        tiles='Stamen Terrain'
    )

    tooltip = 'Click me!'

    folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(viz_map)
    folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>', tooltip=tooltip).add_to(viz_map)

    return viz_map

In [ ]:
intro_to_folium()

## Visualising GeoJson Data

Visualisation of geoJson data in Folium is easy!

In [ ]:
def visualize_geojson_simple(filename, layer_name):
    viz_map = folium.Map()

    folium.GeoJson(
        filename,
        name=layer_name
    ).add_to(viz_map)

    bounding_coords = get_all_coords_in_file(filename)
    viz_map.fit_bounds(bounding_coords)
    folium.LayerControl().add_to(viz_map)
    return viz_map

In [ ]:
visualize_geojson_simple(hawker_centre_geojson, "hawker centre locations")

In [ ]:
@dataclass
class Marker:
    lat: float
    lon: float
    text: str
    image: List[str]
    coord: List[float] = field(init=False)
        
    def __post_init__(self):
        self.coord = [self.lat, self.lon]

In [ ]:
def get_markers_from_file(filename):
    markers = []
    with open(filename, "r") as f:
        features = json.load(f)["features"]
        for feature in features:
            coords = feature["geometry"]["coordinates"][:2][::-1]
            name = feature["properties"]["Name"]
            description = feature["properties"]["Description"]
            markers.append(Marker(coords[0], coords[1], f"Name: {name} <br><br>Description: {description}", []))

    return markers

def visualize_data_with_custom_marker(filename, layer, viz_map=None):
    markers = get_markers_from_file(filename)

    for marker in markers:
        folium.Marker(
            location=marker.coord,
            popup=folium.Popup(marker.text, max_width=600, min_width=600)
        ).add_to(layer)

    if viz_map is None:
        viz_map = folium.Map()
    
    viz_map.add_child(layer)
    bounding_coords = get_all_coords_in_file(filename)
    viz_map.fit_bounds(bounding_coords)
    folium.LayerControl().add_to(viz_map)
    
    return viz_map

In [ ]:
visualize_data_with_custom_marker(hawker_centre_geojson, folium.FeatureGroup("hawker centre locations"))

In [ ]:
visualize_data_with_custom_marker(hawker_centre_geojson, plugins.MarkerCluster(name="hawker centre locations"))